In [84]:
class Deck(object):
    
    def __init__(self):
        self.fullDeck = ["A","1","2","3","4","5","6","7","8","9","10","J","Q","K"]*4
        self.cards =    ["A","1","2","3","4","5","6","7","8","9","10","J","Q","K"]*4
        
    def __len__(self):
        return len(self.cards)
    
    #to take the card out of the deck
    def takecard(self):
        import random
        c=self.cards.pop(self.cards.index(random.choice(self.cards)))
        return c
    #reset deck
    def reset(self):
        self.cards=self.fullDeck
        
    
class Player(object):
    
    def __init__(self,accBal=1000,wins=0,losses=0,draws=0,busts=0,bet=0,hand=[]):
        self.accBal = accBal
        self.wins = wins
        self.losses = losses
        self.draws = draws
        self.busts = busts
        self.bet = bet
        #player hand at any given time
        self.hand=hand
        
    def change_bal(self,amt):
        self.accBal+=amt
        
    def updatecount(self,typ):
        if typ=='win':
            self.wins+=1
        elif typ=='loss':
            self.losses+=1
        elif typ=='draw':
            self.draws+=1
        else:
            self.busts+=1
            
    def printStatus(self):
        print(""" 
          Account Balance: ${}
          number of wins: {}
          number of losses: {}
          number of draws: {}
          number of busts: {}
                    """.format(self.accBal,self.wins,self.losses,self.draws,self.busts))
    def updateHand(self,card):
        self.hand.append(card)
        
    def clearHand(self):
        self.hand=[]
        
    def showHand(self):
        print(self.hand)
        
    def betAmount(self,amt):
        self.bet=amt
        
class Gamestatus(object):
    
    def __init__(self,endgame=False,roundstatus=None):
        self.endgame=endgame
        self.roundstatus=roundstatus
        
    def checkGamestatus(self,player):
        if player.accBal>=3000:
            print("congratulations you have won the game")
            self.endgame=True
        elif player.accBal<=0:
            print("you have lost")
            self.endgame=True
        else:
            self.endgame=False
            
        
    def updateRoundstatus(self,p):
        A=p.count("A");two=p.count("2");three=p.count("3");four=p.count("4");five=p.count("5");six=p.count("6");
        seven=p.count("7");eight=p.count("8");nine=p.count("9");ten=p.count("10");J=p.count("J");Q=p.count("Q");
        K=p.count("K");
            
        total= two*2 + three*3 + four*4 + five*5 + six*6 + seven*7 + eight*8 + nine*9\
                + ten*10 + J*10 + Q*10 + K*10
        
        
        #account for ace being 1 or 11
        if (21>=total + A*1>16) or (21>=total + A*11>16):
            self.roundstatus = "win"
        elif total==16:
            self.roundstatus = "Draw"
        else:
            self.roundstatus = "loss"
        

In [91]:
from IPython.display import clear_output

def initialise():
    deck=Deck()
    player=Player()
    game=Gamestatus()
    player.printStatus()
    
    return deck,player,game

def checkYN(inp):
    while True:
        if inp.lower() in ["yes","no","y","n"]:
            if inp.lower() in ["yes","y"]:
                return "y"
                break
            else:
                return "n"
        else:
            print(" please enter Y or N ,yes or no")
            inp=input()
            
def checkbet(player):
    while True:
        inp=input()
        try:
            if inp.isdigit() and 0<int(inp)<=player.accBal:
                player.betAmount(int(inp))
                break
            else:
                print("enter whole num i.e within your account balance")
        
        except:
            print("enter whole num i.e within your account balance")
            
def hitOrStand():
    while True:
        inp=input()
        if inp.lower() in ["hit","stand"]:
            if inp.lower() == "hit":
                return "hit"
                break
            else:
                return "stand"
        else:
            print(" please enter Hit or Stand")
            continue
            
def checkForBust(p):
    #input will be an array
    A= p.count("A");two=p.count("2");three=p.count("3");four=p.count("4");five=p.count("5");six=p.count("6");
    seven=p.count("7");eight=p.count("8");nine=p.count("9");ten=p.count("10");J=p.count("J");Q=p.count("Q");
    K=p.count("K");
    
    if (A*1 + two*2 + three*3 + four*4 + five*5 + six*6 + seven*7 + eight*8 + nine*9\
        + ten*10 + J*10 + Q*10 + K*10) > 21:
        return True
    else:
        return False
    

def checkHandOk(player):
    if len(player.hand)==5:
        return "limit_reached"
        
    elif checkForBust(player.hand):
        return "bust"
    elif len(player.hand)==5 and checkForBust(player.hand):
        return "bust_limit"
    else:
        return "ok"
    
def updateWinLoss(status,player):
    if status=="win":
        player.updatecount("win")
        print("playr has won this round")
        #update account balance
        player.change_bal(player.bet*2)
        
    elif status=="draw":
        player.updatecount("draw")
        print("playr has drawn this round")
        player.change_bal(player.bet)
        
    else:
        player.updatecount("loss")
        print("playr has lost this round...:( ")
        
def aRound(deck,player,game):
    import time
    while True:
        #this is where blackjack happens
        #check hand in round
        if checkHandOk(player)=="ok":
            
            print("would you like to hit or stand")
            output=hitOrStand()
        
            if output=="Hit":
                print("Dealing your card")
                #get a card from the deck
                card=deck.takecard()
                #put it in players hand
                player.updateHand(card)
                player.showHand()
            
            else:
                #player stand check for win condition
                print("checking your card")
                game.updateRoundstatus(player.hand)
                updateWinLoss(game.roundstatus,player)
            
                break
                
        elif checkHandOk(player)=="bust":
            print("hand is bust.... checking status and updating your profile ")
            game.updateRoundstatus(player.hand)
            updateWinLoss(game.roundstatus,player)
            player.updatecount("bust")
            
            break
                
                
        elif checkHandOk(player)=="limit_reached":
            print("limit of 5 is reached.... checking status")
            game.updateRoundstatus(player.hand)
            updateWinLoss(game.roundstatus,player)
                
            
            break
                
        elif checkHandOk(player)=="bust_limit":
            print("limit of 5 is reached and you have busted.... checking status...")
            game.updateRoundstatus(player.hand)
            updateWinLoss(game.roundstatus,player)
            player.updatecount("bust")
                
            break
            
        else:
            break
            

def gameSession(output,deck,player,game):
    import time
    while True:
        #betting
        print("How much do you want ot bet on this round?")
        checkbet(player)
        #take bet from balance
        player.change_bal(-player.bet)
        
        #initial dealin of the first card
        
        card=deck.takecard()
        #put it in player hand
        player.updateHand(card)
        player.showHand()
        
        aRound(deck,player,game)
        #wait 3secs before clearing the round
        
        time.sleep(3)
        clear_output()
        player.printStatus()
        
        #reset hand and deck
        player.clearHand()
        deck.reset()
        #check game status ,if condtn is True endgame ,if false keep playing
        game.checkGamestatus(player)
        
        if game.endgame== False:
            #need to able to break gamesession early by quitting after round finishes
            print("would you like to play another round? Y/N")
            output=checkYN(input())
            
            if output=="y":
                continue
            else:
                break
                
        else:
            break
            

In [93]:
from IPython.display import clear_output

print("Welcome to Blackjack: To win you must reach 3000$:")
#initialize create new player and deck
deck,player,game=initialise()

print("would you like to start now Y/N?")

output=checkYN(input())
clear_output()
if output=='y':
    print("dealing your first card...")
    
    gameSession(output,deck,player,game)
    
print("Game Exiting...")

 
          Account Balance: $0
          number of wins: 1
          number of losses: 3
          number of draws: 0
          number of busts: 0
                    
you have lost
Game Exiting...
